In [1]:
import pandas as pd
import requests
import json

In [13]:
METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

input_gene_set = {
#     # CMS1 colotype
#     "up_genes": ["ASPHD2", "WARS", "HPSE", "GNLY"],
#     "down_genes": ["AXIN2", "SEMA5A", "CDHR1", "CTTNBP2", "ATP9A", "DACH1"]
    
#     # CMS1 colotype + MLH1
#     "up_genes": ["ASPHD2", "WARS", "HPSE", "GNLY"],
#     "down_genes": ["AXIN2", "SEMA5A", "CDHR1", "CTTNBP2", "ATP9A", "DACH1", 'MLH1']
    
    # CMS1 colotype + GALNT12 + MLH1
    "up_genes": ["ASPHD2", "WARS", "HPSE", "GNLY", 'GALNT12'],
    "down_genes": ["AXIN2", "SEMA5A", "CDHR1", "CTTNBP2", "ATP9A", "DACH1", 'MLH1']

}

all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

payload = {
    "filter": {
        "where": {
            "meta.symbol": {
                "inq": all_genes
            }
        },
        "fields": ["id", "meta.symbol"]
    }
}
res = requests.post(METADATA_API + "entities/find", json=payload)
entities = res.json()

for_enrichment = {
    "up_entities": [],
    "down_entities": []
}

for e in entities:
    symbol = e["meta"]["symbol"]
    if symbol in input_gene_set["up_genes"]:
        for_enrichment["up_entities"].append(e["id"])
    elif symbol in input_gene_set["down_genes"]:
        for_enrichment["down_entities"].append(e["id"])

print(json.dumps(for_enrichment, indent=2))

{
  "up_entities": [
    "0f8b2b5b-b0c7-5405-aca7-c5a016389e87",
    "a5a07ec6-3dfc-464d-af32-5d74fb9c605f",
    "413d4bda-ea73-4741-b8f2-9fdf9af21d70",
    "f4cf4c9f-5ec3-4d34-b103-4f32dc125865",
    "86a70db1-95c0-480b-9648-19b3fe3efb97"
  ],
  "down_entities": [
    "d2b49e7b-d6a4-48d2-9d10-137650b71a89",
    "faefdf14-7ead-4a45-8d1d-65b48ea42b0b",
    "22ddb9d0-8ad1-4969-a364-23b7d0dc5750",
    "ca3299b0-a723-5f0d-ad57-d78820306419",
    "0fec77c0-6e66-4edd-a256-5fb746ae513a",
    "3b4068e2-4753-4575-b046-ae32501b5568",
    "fac29628-8e8f-4949-ae79-fdba9caf9b8f"
  ]
}


In [14]:
payload = {
    "meta": {
        "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
        **for_enrichment
    },
    "type": "signature"
}
res = requests.post(METADATA_API + "user_input", json=payload)
persistent_id = res.json()["id"]
print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/a5984f59-ffed-5689-84c1-ecb5132defd0
